# RAG Process

### 1. 필요한 라이브러리 설치 및 임포트

In [27]:
# 필요한 라이브러리 import
from langchain.document_loaders import PyPDFLoader                 # PDF 문서를 불러오기 위한 로더
from langchain.text_splitter import RecursiveCharacterTextSplitter # 문서를 일정 단위로 쪼개는 도구
from langchain.embeddings import HuggingFaceBgeEmbeddings          # OpenAI 임베딩 모델 사용
from langchain.vectorstores import Chroma                          # 벡터 DB인 Chroma 사용
from langchain_community.llms import Ollama                        # OpenAI GPT 모델 설정
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

### 2. PDF 파일 로딩 및 텍스트 추출

In [2]:
# 텍스트 파일을 문서로 로드
loader1 = PyPDFLoader("../data/squat1.pdf")  # 만약 파일의 이미지까지 불러오고 싶다면 extract_images=True 추가
doc1 = loader1.load()

loader2 = PyPDFLoader("../data/squat2.pdf")
doc2 = loader2.load()

### 3. 문서 조각화 (Chunking)

In [3]:
# 문서 분할 클래스 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
    )
split_docs = text_splitter.split_documents(doc1 + doc2)

### 4. 임베딩 모델 설정 및 ChromaDB에 임베딩 저장

In [4]:
# OpenAI Embeddings()
embedding_model = HuggingFaceBgeEmbeddings()

# Chroma 벡터 저장소에 문서 임베딩 저장
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    persist_directory="../chroma_db/pdf_docs")

# 로컬에 저장
vectorstore.persist()

C:\Users\user\AppData\Local\Temp\ipykernel_6712\396006598.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings()
C:\Users\user\AppData\Local\Temp\ipykernel_6712\396006598.py:2: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  embedding_model = HuggingFaceBgeEmbeddings()
C:\Users\user\AppData\Local\Temp\ipykernel_6712\396006598.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer suppo

### 5. retirever 검색기 설정 및 테스트

In [ ]:
# 저장된 Chroma 벡터 DB 로드
vectorstore = Chroma(
    persist_directory="../chroma_db/pdf_docs",
    embedding_function=embedding_model
    )

# Retriever 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",           # 유사도 분석 기반
    search_kwargs={"k": 5}              # 3개의 문장만 갖고오기
    )

C:\Users\user\AppData\Local\Temp\ipykernel_6712\1071343833.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


### 6. LLM 설정

In [25]:
# Local LLM Model (Ollama: llama3)
llm = Ollama(model="llama3", temperature=0)          # temperature: 창의성 레벨 (0~1)

### 7. RAG Chain 설정

In [31]:
memory = ConversationBufferMemory()

rag_chain = ConversationChain(
    llm=llm,
    memory=memory,
    # chain_type="stuff",                 # stuff: 여러 문서를 단순히 이어붙여서 LLM에 전달하는 방식
    # chain_type="map_reduce",            # map_reduce: 각 문서를 개별적으로 요약하고, 마지막에 LLM이 종합해서 답변 생성하는 방식
    # retriever=retriever,
    verbose=True      # 참조 문서를 결과에 포함
)

### 8. Query Test

In [36]:
query = "스쿼트할 때 척추 중립을 유지해야 하는 이유를 알려줘"

# RAG 체인에 질문 전달
response = rag_chain.predict(input=query)

# 답변 출력
print("📘 Llama3 응답:")
# print(response['result'])
print(response)

# 참조 문서 출력 (선택)
# print("\n📄 참조 문서:")
# print(response['source_documents'][1].page_content)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 스쿼트할 때 척추 중립을 유지해야 하는 이유를 알려줘
AI:

> Finished chain.
📘 Llama3 응답:
A great question! 😊

When performing squats, maintaining a neutral spine position is crucial for several reasons. Firstly, when the spine is in a neutral position, it allows for optimal activation of the gluteal muscles, which are responsible for hip extension and external rotation during the squat movement. This ensures proper engagement of the glutes, reducing the risk of overusing or underusing them.

Secondly, a neutral spine helps to maintain proper pelvic alignment, which is essential for maintaining good posture and preventing excessive strain on the lower back. When the pelvis is in a neu